# Worksheet 08

Name:  Rishab Sudhir

UID: U64819615

### Topics

- Soft Clustering
- Clustering Aggregation

### Probability Review

Read through [the following](https://medium.com/@gallettilance/overview-of-probability-3272b72c82c8)

### Soft Clustering

We generate 10 data points that come from a normal distribution with mean 5 and variance 1.

In [1]:
import random
import numpy as np
from sklearn.cluster import KMeans

mean = 5
stdev = 1

s1 = np.random.normal(mean, stdev, 10).tolist()
print(s1)

[4.571039268782286, 6.67399196263077, 5.14999345864837, 7.200851025315503, 5.523359298318354, 4.960830730449355, 6.108785483867473, 4.068812826445038, 4.3599258320220615, 3.4839551252496195]


a) Generate 10 more data points, this time coming from a normal distribution with mean 8 and variance 1.

In [2]:
s2 = np.random.normal(8,1,10).tolist() #mean stdec, how many data points
print(s2)

[8.08623957273794, 8.765639171106821, 8.058587117211337, 8.159295960861582, 7.496941177205167, 5.8118178104831, 9.501706033633328, 9.40376240876326, 9.296199090114687, 7.239679332447352]


b) Flip a fair coin 10 times. If the coin lands on H, then pick the last data point of `s1` and remove it from `s1`, if T then pick the last data point from `s2` and remove it from `s2`. Add these 10 points to a list called `data`.

In [3]:
data = []
for i in range(10):
    # flip coin
    coin_output = random.choice([0, 1])
    if coin_output == 0:
        p1 = s1.pop()
        data.append(p1)
    else:
        p2 = s2.pop()
        data.append(p2)
print(data)

[7.239679332447352, 3.4839551252496195, 4.3599258320220615, 9.296199090114687, 9.40376240876326, 4.068812826445038, 9.501706033633328, 6.108785483867473, 4.960830730449355, 5.523359298318354]


c) This `data` is a Gaussian Mixture Distribution with 2 mixture components. Over the next few questions we will walk through the GMM algorithm to see if we can uncover the parameters we used to generate this data. First, please list all these parameters of the GMM that created `data` and the values we know they have.

The prior pr of being species 1

The prior pr of being species 2

The mean and var of the weight species 1

The mean and var of the weight species 2

3 k parameters for k species

d) Let's assume there are two mixture components (note: we could plot the data and make the observation that there are two clusters). The EM algorithm asks us to start with a random `mean_j`, `variance_j`, `P(S_j)` for each component j. One method we could use to find sensible values for these is to apply K means with k=2 here.

1. the centroids would be the estimates of the `mean_j`
2. the intra-cluster variance could be the estimate of `variance_j`
3. the proportion of points in each cluster could be the estimate of `P(S_j)`

Go through this process and list the parameter estimates it gives. Are they close or far from the true values?

In [81]:
kmeans = KMeans(2, init='k-means++').fit(X=np.array(data).reshape(-1, 1))

s1 = [x[0] for x in filter(lambda x: x[1] == 0, zip(data, kmeans.labels_))]
print(s1)
s2 = [x[0] for x in filter(lambda x: x[1] == 1, zip(data, kmeans.labels_))]
print(s2)

prob_s = [ len(s1) / (len(s1) + len(s2)) , len(s2) / (len(s1) + len(s2)) ]
mean = [ sum(s1)/len(s1) , sum(s2)/len(s2) ]
var = [ sum(map(lambda x : (x - mean[0])**2, s1)) / len(s1) , sum(map(lambda x : (x - mean[1])**2, s2)) / len(s2) ]

print("P(S_1) = " + str(prob_s[0]) + ",  P(S_2) = " + str(prob_s[1]))
print("mean_1 = " + str(mean[0]) + ",  mean_2 = " + str(mean[1]))
print("var_1 = " + str(var[0]) + ",  var_2 = " + str(var[1]))

[3.4839551252496195, 4.3599258320220615, 4.068812826445038, 6.108785483867473, 4.960830730449355, 5.523359298318354]
[7.239679332447352, 9.296199090114687, 9.40376240876326, 9.501706033633328]
P(S_1) = 0.6,  P(S_2) = 0.4
mean_1 = 4.7509448827253165,  mean_2 = 8.860336716239656
var_1 = 0.7846450470612045,  var_2 = 0.8807931122973188


/Users/rsudhir/anaconda3/lib/python3.11/site-packages/sklearn/cluster/_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


e) For each data point, compute `P(S_j | X_i)`. Comment on which cluster you think each point belongs to based on the estimated probabilities. How does that compare to the truth?

In [86]:
from scipy.stats import norm

# TODO: might need to change var to std

prob_s0_x = [] # P(S_0 | X_i)
prob_s1_x = [] # P(S_1 | X_i)
prob_x = [] # P(X_i)

k = 2

for p in data:
    print("point = ", p)
    pdf_i = []

    for j in range(k):
        # P(X_i | S_j)
        pdf_i.append(norm.pdf(p, mean[j], var[j]))
        print("probability (pdf at that particular weight value) of observing that point if it came from cluster " + str(j) + " = ", pdf_i[j])
        # P(S_j) already computed
        prob_s[j]

    #pdf_i[0] is the p(p | s_0)

    # P(X_i) = P(S_0)P(X_i | S_0) + P(S_1)P(X_i | S_1)
    prob_x = prob_s[0] * pdf_i[0] + prob_s[1] * pdf_i[1]

    # P(S_j | X_i) = P(X_i | S_j)P(S_j) / P(X_i)
    prob_s0_x.append( pdf_i[0] * prob_s[0] / prob_x)
    prob_s1_x.append( pdf_i[1] * prob_s[1] / prob_x)

probs = zip(data, prob_s0_x, prob_s1_x)
for p in probs:
    print(p[0])
    print("Probability of coming from S_1 = " + str(p[1]))
    print("Probability of coming from S_2 = " + str(p[2]))
    print()


point =  7.239679332447352
probability (pdf at that particular weight value) of observing that point if it came from cluster 0 =  0.0033240832291170125
probability (pdf at that particular weight value) of observing that point if it came from cluster 1 =  0.08334205784367366
point =  3.4839551252496195
probability (pdf at that particular weight value) of observing that point if it came from cluster 0 =  0.13805668738140373
probability (pdf at that particular weight value) of observing that point if it came from cluster 1 =  3.675577305108128e-09
point =  4.3599258320220615
probability (pdf at that particular weight value) of observing that point if it came from cluster 0 =  0.4490659737468942
probability (pdf at that particular weight value) of observing that point if it came from cluster 1 =  9.704632429093766e-07
point =  9.296199090114687
probability (pdf at that particular weight value) of observing that point if it came from cluster 0 =  2.6281439344114735e-08
probability (pdf at t

f) Having computed `P(S_j | X_i)`, update the estimates of `mean_j`, `var_j`, and `P(S_j)`. How different are these values from the original ones you got from K means? briefly comment.

In [91]:
prob_c = [sum(prob_s0_x)/ len(prob_s0_x), sum(prob_s1_x)/ len(prob_s1_x) ]
mean = [sum([x[0] * x[1] for x in zip(prob_s0_x, data)]) / sum(prob_s0_x), sum([x[0] * x[1] for x in zip(prob_s1_x, data)]) / sum(prob_s1_x) ]
var = [ sum([x[0] * (x[1]-mean[0])**2 for x in zip(prob_s0_x, data)]) / sum(prob_s0_x) , sum([x[0] * (x[1]-mean[1])**2 for x in zip(prob_s1_x, data)]) / sum(prob_s1_x)]

print("P(S_1) = " + str(prob_s[0]) + ",  P(S_2) = " + str(prob_s[1]))
print("mean_1 = " + str(mean[0]) + ",  mean_2 = " + str(mean[1]))
print("var_1 = " + str(var[0]) + ",  var_2 = " + str(var[1]))

P(S_1) = 0.6,  P(S_2) = 0.4
mean_1 = 4.7696761704713,  mean_2 = 8.869032137794662
var_1 = 0.8314402262012581,  var_2 = 0.8912729496473333


g) Update `P(S_j | X_i)`. Comment on any differences or lack thereof you observe.

h) Use `P(S_j | X_i)` to create a hard assignment - label each point as belonging to a specific cluster (0 or 1)